## IMPORT

In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import scipy
from scipy.optimize import fmin_l_bfgs_b 
from cvxopt import matrix, solvers
import pickle as pkl
from scipy import optimize
from scipy.linalg import cho_factor, cho_solve

In [4]:
Xtr = np.array(pd.read_csv('data_image/Xtr.csv',header=None,sep=',',usecols=range(3072))) 
Xte = np.array(pd.read_csv('data_image/Xte.csv',header=None,sep=',',usecols=range(3072))) 
Ytr = np.array(pd.read_csv('data_image/Ytr.csv',sep=',',usecols=[1])).squeeze() 

## VIZUALISATION

Let's visualize data :

In [ ]:
def plot_images_grid(data, nrows, ncols):
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*2, nrows*2))
    random=np.random.choice(data.shape[0],size=nrows*ncols)
    #data=(data-np.min(data))/(np.max(data)-np.min(data))
    for j, ax in enumerate(axes.flat):
        i=random[j]
        if i < data.shape[0]:
            image_data = data[i, :]
            
            # Normaliser les données dans l'intervalle [0, 1]
            # min_val = image_data.min()
            # max_val = image_data.max()
            # print(min_val,max_val)
            # image_data = (image_data - min_val) / (max_val - min_val)
            # print(image_data.max())
            
            red_channel = image_data[:1024].reshape((32, 32))
            red_channel=(red_channel-red_channel.min())/(red_channel.max()-red_channel.min())
            green_channel = image_data[1024:2048].reshape((32, 32))
            green_channel=(green_channel-green_channel.min())/(green_channel.max()-green_channel.min())
            blue_channel = image_data[2048:].reshape((32, 32))
            blue_channel=(blue_channel-blue_channel.min())/(blue_channel.max()-blue_channel.min())
            image = np.stack((red_channel, green_channel, blue_channel), axis=-1)

            ax.imshow(image)
            ax.axis('off')
        else:
            ax.axis('off')

    plt.tight_layout()
    plt.show()
plot_images_grid(Xtr,2,2)

## KERNELS

Let's define some kernels :

In [13]:
class RBF():
    """
    Compute the matrix of the Gaussian Kernel : 
    
    (K)_ij=K(X_i,Y_j)=exp( 1/2*sigma^2 * ||X_i-Y_j||^2 ) the Gaussian Kernel evaluated between the ith data and jth data

    Parameters : 
    X : 2d array size (n,p) 
        the Data matrix with n the number of data and p the size of data
    Y : 2d array size (q,p) 
        the Data matrix with q the number of data and p the size of data
    sigma : float 
             Variance of the GaussianKernel 

    Outputs :
    2d array size (n,q)
    Matrix of the Gaussian Kernel
    """   
    def __init__(self, sigma=1.):
        self.sigma = sigma  ## the variance of the kernel
    def kernel(self,X,Y):
        ## Input vectors X and Y of shape Nxd and Mxd
        diff2 = np.sum(X**2, axis=1)[:, None] + np.sum(Y**2, axis=1)[None, :] - 2 * np.dot(X, Y.T)
        return  np.exp(-diff2/(2*self.sigma**2)) ## Matrix of shape NxM

In [14]:
class polynomial():
    """
    Compute the matrix of the Polynomial Kernel : 
    
    (K)_ij=K(X_i,Y_j)=(<X_i,Y_j>)^d the Polynomial Kernel of degree d evaluated between the ith data and jth data

    Parameters : 
    X : 2d array size (n,p) 
        the Data matrix with n the number of data and p the size of data
    Y : 2d array size (q,p) 
        the Data matrix with q the number of data and p the size of data
    d : Integer
            Degree of the polynomial kernel

    Outputs :
    2d array size (n,q)
    Matrix of the Gaussian Kernel
    """
    def __init__(self,d=2):
        self.d=d

    def kernel(self,X,Y):
        return np.dot(X,Y.T) ** self.d
        # X_intercept = np.concatenate((X,np.ones(X.shape[0]).reshape(-1,1)), axis=1)
        # Y_intercept= np.concatenate((Y,np.ones(Y.shape[0]).reshape(-1,1)), axis=1)
        # return np.sum(X_intercept * Y_intercept[:,None,:], axis=-1) ** self.d

## MODELS

In [ ]:
class Kernel_ridge_reg():
    """
    Class which comput the solution to the Kernel Ridge Regression with regularization parameter lambda (lmbda)

    ----> We have a close form for the solution : 
            alpha=(K+lambda*n*I)^-1y
            pred=K@alpha
    """

    def __init__(self,lmbda):
        self.lmbda=lmbda

    def train(self,K,y):
        mat=K+self.lmbda*K.shape[0]*np.identity(K.shape[0])
        self.alpha=scipy.linalg.solve(mat,y)
    
    def fit(self,K):
        print(self.alpha.shape,K.shape)
        return self.alpha@K

In [ ]:
class Kernel_logistic_reg():
    """
    Class which comput the solution to the Kernel logistic Regression with regularization parameter lambda (lmbda)

    ----> We have to solve the smooth convex opti problem : 
            alpha=argmin 1/n sum_i=1^n logisitic(y_i[Kalpha]_i)+lambda/2 *alpha^T@K@alpha

          We solve this with the L-FBGS form scipy

          and then : 
            pred=K@alpha
    """

    def __init__(self,lmbda,alpha0):
        self.lmbda=lmbda
        self.alpha0=alpha0

    def obj(self,K,y,alpha):
        n=len(K)
        return np.sum(np.log(1+np.exp(-y*(K@alpha))))/n+self.lmbda*alpha.T@K@alpha/2

    def derivative(self,K,y,alpha):
        n=len(K)
        P=-np.diag(1/(1+np.exp(y*(K@alpha))))
        return K@P@y/n+self.lmbda*K@alpha

    def train(self,K,y):
        ob= lambda alpha : self.obj(K=K,y=y,alpha=alpha)
        der=lambda alpha : self.derivative(K=K,y=y,alpha=alpha)
        alpha,_,_=fmin_l_bfgs_b(ob, self.alpha0, der, args=(), pgtol=1e-50, factr =1e-50)
        self.alpha=alpha
    
    def fit(self,K):
        return K@self.alpha

In [ ]:
class SVM():
     """
    Class which train SVM models with Kernels methods takes a dataset with labels in {-1,1} 

    """
     def __init__(self,lmbda):
          self.lmbda = lmbda

     def train(self,K,y):
          """ 
          We want to solve max mu^T@1 - 1/4*lambda mu^t@diag(y)@K@diag(y)@mu for 0<=mu<=1/n
          and then we have :
               alpha=diag(y)@mu/2*lambda

          We use the package cvxopt wich solve : 
               min 1/2x^T@P@x + q^T@x for Gx<=h and Ax=b
          """
          n=len(K)

          q = -np.ones(n)
          P = np.diag(y) @ K @np.diag(y) / (2*self.lmbda)
          G = np.concatenate((np.identity(n),-np.identity(n)),axis=0)
          h = np.concatenate((np.ones(n)/n , np.zeros(n)),axis=0)[:,None]

          mu=solvers.qp(P=matrix(P),q=matrix(q),G=matrix(G),h=matrix(h))['x']
          
          self.alpha=np.diag(y)@mu / (2*self.lmbda)

     
     def fit(self,K):
          return np.sign(K@self.alpha)
     
     def pred_prob(self,K):
          return K@self.alpha
     


In [11]:
class KernelSVC:

    """
    Class which train SVM models with Kernels methods takes a dataset with labels in {-1,1} 

    """
    
    def __init__(self, C, kernel, epsilon = 1e-1):
        self.type = 'non-linear'
        self.C = C                               
        self.kernel = kernel        
        self.alpha = None
        self.support = None
        self.epsilon = epsilon 
        self.norm_f = None
       
    
    def fit(self, X, y):
        N = len(y)
        K=self.kernel(X,X)
        print('kernel computed')
        diag=np.diag(y)

        # Lagrange dual problem
        def loss(alpha):
            return  (1/2)*(diag@alpha).T@K@(diag@alpha)-np.sum(alpha) 

        # Partial derivate of Ld on alpha
        def grad_loss(alpha):
            return diag@K@diag@alpha-np.ones_like(alpha) 


        # Constraints on alpha of the shape :
        # -  d - C*alpha  = 0
        # -  b - A*alpha >= 0

        fun_eq = lambda alpha: alpha.T@y  # '''----------------function defining the equality constraint------------------'''        
        jac_eq = lambda alpha: y   #'''----------------jacobian wrt alpha of the  equality constraint------------------'''
        fun_ineq = lambda alpha: np.concatenate((alpha,self.C-alpha))   # '''---------------function defining the inequality constraint-------------------'''     
        jac_ineq = lambda alpha:  np.concatenate((np.identity(len(alpha)),-np.identity(len(alpha)))) # '''---------------jacobian wrt alpha of the  inequality constraint-------------------'''
        
        constraints = ({'type': 'eq',  'fun': fun_eq, 'jac': jac_eq},
                       {'type': 'ineq', 
                        'fun': fun_ineq , 
                        'jac': jac_ineq})
        print('begin opti :')
        optRes = optimize.minimize(fun=lambda alpha: loss(alpha),
                                   x0=np.ones(N), 
                                   method='SLSQP', 
                                   jac=lambda alpha: grad_loss(alpha), 
                                   constraints=constraints,tol=self.epsilon)
        self.alpha = optRes.x
        print('end opti')
        ## Attributes
        print('begin operation')
        indice_sv=np.where(np.abs(self.alpha)>1e-5)[0]
        self.support=X[indice_sv]
        self.alpha_support=self.alpha[indice_sv]
        print('1')
        self.beta=diag@self.alpha
        self.beta_support=self.beta[indice_sv]
        print('2')
        margin_indices = np.where((self.alpha > 1e-5) & (self.alpha < self.C-1e-5))[0]
        self.margin_points = X[margin_indices]  #'''------------------- A matrix with each row corresponding to a point that falls on the margin ------------------'''
        print('3')
        self.b = np.mean(y[indice_sv]-(K@self.beta)[indice_sv])  #''' -----------------offset of the classifier------------------ '''
        print('4')
        self.norm_f = self.beta[indice_sv].T@(K@self.beta)[indice_sv]# '''------------------------RKHS norm of the function f ------------------------------'''

 
    def separating_function(self,x):
        # Input : matrix x of shape N data points times d dimension
        # Output: vector of size N
        return self.kernel(x,self.support)@self.beta_support
    
    
    def predict(self, X):
        """ Predict y values in {-1, 1} """
        d = self.separating_function(X)
        return 2 * (d+self.b> 0) - 1

## UTILS FUNCTION

In [ ]:
def create_reduce_dataset(X,Y,size):
    """ 
    Create a dataset of size = 10*size with 10% of each class
    
    and transform the labels of class k in 1 and the labels of the others classes in -1 
    """

    Y_new=np.array([])
    X_new=np.random.randint(2,size=(1,X.shape[1]))

    for i in range(len(np.unique(Y))):
        ind=np.random.choice(np.array(np.where(Y==i))[0],size=size)
        X_new=np.concatenate((X_new,X[ind]))
        Y_new=np.concatenate((Y_new,i*np.ones(size)))
    X_new=X_new[1:]
    ind=np.arange(Y_new.shape[0])
    np.random.shuffle(ind)
    Y_shuffled=Y_new[ind]
    X_shuffled=X_new[ind]
    return X_shuffled,Y_shuffled


In [ ]:
def create_reduce_dataset_onevsall(X,Y,k,size):
    """ 
    Create a dataset of size = 18*size with 50% of class k and 50% random classes  
    
    and transform the labels of class k in 1 and the labels of the others classes in -1 
    """

    Y_new=np.array([])
    X_new=np.random.randint(2,size=(1,X.shape[1]))

    for i in range(len(np.unique(Y))):
        if i == k :
            ind=np.random.choice(np.array(np.where(Y==i))[0],size=size*9)
        else :
            ind=np.random.choice(np.array(np.where(Y==i))[0],size=size)
        X_new=np.concatenate((X_new,X[ind]))
        if i==k:
            Y_new=np.concatenate((Y_new,np.ones(size*9)))
        else :
            Y_new=np.concatenate((Y_new,-np.ones(size)))
            
    X_new=X_new[1:]
    ind=np.arange(Y_new.shape[0])
    np.random.shuffle(ind)
    Y_shuffled=Y_new[ind]
    X_shuffled=X_new[ind]
    return X_shuffled,Y_shuffled

In [ ]:
def accuracy(y_pred,y_test):
    return np.sum(y_pred==y_test)/len(y_pred)

## RUN CODE

Let's try with the class k=4 and a dataset of size train : 600 and test : 300 

The goal is that the classifier classify well the class k from the others class 

In [ ]:
X,Y=create_reduce_dataset_onevsall(Xtr,Ytr,k=4,size=50)
X_train=X[:600]
X_test=X[600:900]
Y_train=Y[:600]
Y_test=Y[600:900]
Big_X=np.concatenate((X_train,X_test),axis=0)

Let's compute the Grams Matrix (let's try with polynomial kernel), it takes a lot of time (1min40 for 900x900 kernel) : 

In [ ]:
K = polynomial().kernel(Big_X,X_train)
K_train = K[:len(X_train),:len(X_train)]
K_test = K[:,len(X_train):]

In [ ]:
K_train.shape,K_test.shape

Let's try differents models : 

In [ ]:
# RIDGE REG not very appropriated for the problem (classification != regression )
classifier=Kernel_ridge_reg(lmbda=0.1)

classifier.train(K_train,Y_train) 

Y_pred = classifier.fit(K_test)

accuracy(np.sign(Y_pred),Y_test)

In [ ]:
# Logistic REG 
classifier=Kernel_logistic_reg(lmbda=0.1,alpha0=np.random.rand(len(X_train)))

classifier.train(K_train,Y_train)

Y_pred = classifier.fit(K_test.T)

accuracy(np.sign(Y_pred),Y_test)

In [ ]:
classifier=SVM(lmbda=0.2)

classifier.train(K_train,Y_train)

Y_pred = classifier.fit(K_test.T)
print(classifier.alpha)

accuracy(Y_pred[:,0],Y_test)

In [ ]:
K=RBF().kernel
classifier=KernelSVC(C=1,kernel=K)
classifier.fit(X_train,Y_train)
Y_pred=classifier.predict(X_test)
accuracy(Y_pred,Y_test)

Let's try to do a loop and attributing an SVM for all classes : 
ONE VS REST strategy 
ONE VS ONE strategy 

### NYSTROM APPROXIMATION

https://github.com/fredhallgren/nystrompca/blob/develop/nystrompca/algorithms/nystrom_KPCA.py

In [ ]:
class NystromKPCA():
    """
    Compute an approximation of kernel with the PCA techniques, 
    taking m data points, projecting on p directions (p<=m)

    """
    def __init__(self,kernel,p,m) :
       self.kernel=kernel
       self.m=m
       self.p=p
       self.alphas=None
       self.big_approximated_kernel=None
       self.big_approximated_repr=None
       self.X_subset=None

    def fit_PCA(self, X):
        # assigns the vectors
        _=self.choose_subset(X)

        K=self.kernel(self.X_subset,self.X_subset)

        #We have to center the kernel
        #center = (I-U)K(I-U)
        
        U=np.ones(K.shape)
        I=np.eye(K.shape[0])
        K=(I-U)@K@(I-U)
        
        valp,vectp=np.linalg.eigh(K)

        #we take the last r eingenvalues
        lmbda=valp[-self.p:]
        inverse=1/np.sqrt(lmbda)

        self.alphas=(inverse*vectp[:,-self.p:]).T
    

    def approximated_repr(self,x):
        return self.alphas@self.kernel(self.X_subset,x)

    def appro_kernel(self,X,Y):
        repr_X=self.alphas@self.kernel(self.X_subset,X)
        repr_Y=self.alphas@self.kernel(self.X_subset,Y)
        return repr_X.T@repr_Y

    def choose_subset(self,X):
        self.n=X.shape[0]
        set=np.random.choice(self.n,self.m)
        self.X_subset=X[set]
        return set

In [10]:
def create_dataset_onevsall(Y,k):
    """  
    Transform the labels of class k in 1 and the labels of the others classes in -1 
    """
    Y_onevall=-np.ones_like(Y)
    Y_onevall[Y==k]=1
    return Y_onevall

In [15]:
kernel=RBF().kernel
classifier=KernelSVC(C=1,kernel=kernel)
classifier.fit(Xtr,Ytr)

kernel computed
begin opti :
